**извлечение признаков из текста на естественном языке**

классификатор текстов

частотный анализ с очисткой стоп-слов (TF)

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
2549,"\r\nЛетом прошлого года астрономы зафиксировали необычно яркую вспышку на расстоянии в 4 миллиарда световых лет от Солнечной системы. Было заявлено, что во Вселенной родилась самая яркая в истории...",science
1440,"Бывший губернатор Сахалинской области Александр Хорошавин и члены его семьи — жена и совершеннолетний сын — оспаривают антикоррупционный закон ""О контроле за соответствием расходов лиц, замещающих...",politics


##  CountVectorizer + TF

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
stopwords= [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 
    'так', 'его', 'но',  'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 
    'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 
    'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или',  'ни', 'быть', 'был', 'него', 'до', 
    'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 
    'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем',
    'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 
    'ж', 'тогда', 'кто','этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 
    'этом', 'один', 'почти', 'мой', 'тем','чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем',
    'всех', 'никогда', 'можно', 'при', 'наконец', 'два','об', 'другой', 'хоть', 'после', 
    'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них','какая', 'много', 
    'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда',
    'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 
    'всегда', 'конечно', 'всю', 'между']

In [8]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    # tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in stopwords ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов
    return ' '.join( [ t.strip() for t in tt if t ] )

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# ручная очистка
tf = TfidfVectorizer(
    preprocessor=preprocessor,
    lowercase=False,
    stop_words=None,
    use_idf=False,
    norm='l2')

tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7fb50d3cd378>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=False, vocabulary=None)

In [10]:
len(tf.vocabulary_)

44173

## формируем датасеты

In [11]:
X = tf.transform( data['text'] )
X.shape

(3196, 44173)

In [12]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [13]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

---

In [14]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [15]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.9, random_state=get_seed() )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((319, 44173), (319,), (2877, 44173), (2877,))

## обучаем

In [16]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000)
clf.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

## тестируем

In [17]:
o = clf.predict(X_train)

In [18]:
accuracy_score(y_train,o)

1.0

In [19]:
# print( classification_report(y_train,o) )

---

In [20]:
o = clf.predict(X_test)

In [21]:
accuracy_score(y_test,o)

0.7400069516857838

In [22]:
print( classification_report(y_test,o) )

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       222
           1       0.71      0.85      0.77       313
           2       0.68      0.61      0.65       249
           3       0.74      0.27      0.40        85
           4       0.80      0.84      0.82       399
           5       0.77      0.86      0.82       542
           6       0.82      0.24      0.37        58
           7       0.00      0.00      0.00        50
           8       0.82      0.77      0.80       208
           9       0.37      0.24      0.29       130
          10       0.75      0.94      0.83       332
          11       0.64      0.71      0.67       254
          12       0.00      0.00      0.00        35

   micro avg       0.74      0.74      0.74      2877
   macro avg       0.61      0.55      0.56      2877
weighted avg       0.71      0.74      0.72      2877



/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---

In [23]:
from matplotlib import pyplot as plt
import itertools

classes = sorted(labels.keys())
cm = confusion_matrix(y_test,o)
tick_marks = np.arange(len(classes))

plt.figure(figsize=(10,9))

plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.title('Confusion matrix')
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()

plt.show()

<Figure size 1000x900 with 2 Axes>

---

In [24]:
o = clf.predict(X)

In [25]:
labels_inv = { labels[k]:k for k in labels}
# labels_inv

In [26]:
i = np.random.randint(len(data))
print('tag:',data.iloc[i,1])
print('predict:',labels_inv[o[i]])
print('- - - - - - - - - - - - - - - - - - \n')
print(data.iloc[i,0])


tag: politics
predict: politics
- - - - - - - - - - - - - - - - - - 

277

Республиканец Митт Ромни, который еще во время президентской гонки 2012 года называл Российскую Федерацию главным врагом Соединенных Штатов, похоже, все-таки не станет главой американского внешнеполитического ведомства при президентстве Дональда Трампа.

В понедельник, 12 декабря, Ромни выступил с заявлением, из которого американские журналисты сделали вывод, что новоиспеченный хозяин Белого дома Дональд Трамп больше не рассматривает его кандидатуру на пост госсекретаря США, передает "Российский Диалог" со ссылкой на CNN.

Заявление республиканца последовало практически сразу после того, как Трамп объявил, что во вторник утром объявит кандидата на пост главы внешнеполитического ведомства.

Ранее СМИ сообщали, что больше всех шансов возглавить Госдеп имеет бизнесмен Рекс Тиллерсон, который является критиком антироссийских санкций.

В Москве считают, что назначение Тиллерсона может направить российско-американские

---

In [27]:
# def preprocessor(text):
#     tt = [ t for t in text.split() if t ]
#     tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
#     tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
#     tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
#     tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
#     tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
#     return ' '.join( [ t.strip() for t in tt if t ] )
    
# from sklearn.feature_extraction.text import CountVectorizer
# stemmer = Stemmer('russian')
# analyzer = CountVectorizer().build_analyzer()
# def stemmed_words(doc): return [ Stemmer('russian').stemWord(w) for w in analyzer(doc) ]
# vect = CountVectorizer(lowercase=True,preprocessor=preprocessor,analyzer=stemmed_words)
# vect.fit( data['text'] )

# # размер словаря
# print(len(vect.vocabulary_))


# from sklearn.feature_extraction.text import TfidfTransformer
# tf = TfidfTransformer(use_idf=False, norm='l2')
# data_vect = vect.transform( data['text'] )
# tf.fit( data_vect )
